In [1]:
import os
import pickle
import scipy.io
import numpy as np
from scfw.scopt import scopt
from scfw.frank_wolfe import frank_wolfe
import scfw.portfolio as pr
import matplotlib.pyplot as plt

In [2]:
results_folder = './results/'

In [3]:
data_folder = '../data'

problem_name = 'syn_1000_1500_10_50'
data = scipy.io.loadmat(os.path.join(data_folder, problem_name + '.mat'))
R = data['W']
results = {problem_name: {}}

In [4]:
np.random.seed(42)

## Data preprocessing and parameters initialization

In [5]:
N, n = R.shape
Mf = 2
nu = 3
sigma_f = 1
diam_X = np.sqrt(2)
rho = np.sqrt(n)

I = np.eye(n)
Iv = np.ones(n)

#running parameters
x0 = np.ones(n) / n
terminate_tol = 1e-20

#parameters for FW
FW_params={
    'iter_FW':50000,
    'line_search_tol':1e-10,
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'backtracking',#,'estimate', #estimate L
    'use_two_phase':False,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol':1e-5,
    'conj_grad_iter':1000,
}

## Auxilary functions

In [6]:
func_x = lambda x: pr.portfolio(R,x)
grad_x = lambda x, dot_product: pr.grad_portfolio(R, x, dot_product)
grad_beta = lambda x, s, beta, dot_product, dot_product_s: pr.grad_portfolio(R, (1 - beta) * x + beta * s, (1 - beta) * dot_product + beta * dot_product_s)
hess_x = lambda x, dot_product: pr.hess_portfolio(R, x, dot_product)
hess_mult_x = lambda x, dot_product: pr.hess_mult_portfolio(R, x, dot_product)
hess_mult_vec_x = lambda s, dot_product: pr.hess_mult_vec(R, s, dot_product)
extra_func = lambda x: R @ x
linear_oracle = lambda grad: pr.linear_oracle_simplex(grad)
llo_oracle = lambda x, r, grad: pr.llo_oracle(x, r, grad, rho=rho)
prox_func = lambda x, L: pr.proj_simplex(x)

In [7]:
points = [x0]
 
for _ in range(4):
    x = np.abs(np.random.rand(len(x0)))
    x = x / sum(x)
    points.append(x)

## Run FW

In [8]:
run_alpha_policies = ["standard","line_search","icml"]
#run_alpha_policies = ['lloo']


for policy in run_alpha_policies:
    x, alpha_hist, Gap_hist, Q_hist, time_hist, grad_hist = frank_wolfe(func_x,
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       linear_oracle,                                                    
                       x0,
                       FW_params,
                       hess=hess_x,
                       rho=rho,
                       sigma_f=sigma_f,
                       diam_X=diam_X,   
                       lloo_oracle=llo_oracle,                                                 
                       alpha_policy=policy,                                                    
                       eps=terminate_tol, 
                       print_every=10000, 
                       debug_info=False)
    
    results[problem_name][policy] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Gap_hist': Gap_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
        'grad_hist': grad_hist
    }

********* Algorithm starts *********
iter = 1, stepsize = 0.6666666666666666, criterion = 9.999999999999999e-11, upper_bound=0.02128931286911006, lower_bound=-4.917163298675453
iter = 10000, stepsize = 0.0001999600079984003, criterion = 9.999999999999999e-11, upper_bound=-4.475092844319556, lower_bound=-4.475118973650115
iter = 20000, stepsize = 9.999000099990002e-05, criterion = 9.999999999999999e-11, upper_bound=-4.475092910483785, lower_bound=-4.475101714730176
iter = 30000, stepsize = 6.666222251849876e-05, criterion = 9.999999999999999e-11, upper_bound=-4.475092922681078, lower_bound=-4.475098189901174
iter = 40000, stepsize = 4.999750012499375e-05, criterion = 9.999999999999999e-11, upper_bound=-4.475092927022443, lower_bound=-4.475097962961407
iter = 50000, stepsize = 3.999840006399744e-05, criterion = 9.999999999999999e-11, upper_bound=-4.475092929028327, lower_bound=-4.475097253048681
222.2299404144287
********* Algorithm starts *********
iter = 1, stepsize = 0.999999999941792

In [9]:
#with open(os.path.join(results_folder, 'portfolio', problem_name + '_FW.pckl'), 'wb') as f:
#    pickle.dump(results, f)

## Run SCOPT

In [9]:
x, alpha_hist, Q_hist, time_hist, grad_hist = scopt(func_x,
        grad_x,
        hess_mult_x,
        hess_mult_vec_x,
        Mf,
        nu,
        prox_func,
        x0,  
        sc_params,                                              
        eps=terminate_tol,                                              
        print_every=1)
    
results[problem_name]['scopt'] = {
    'x': x,
    'alpha_hist': alpha_hist,
    'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
    'grad_hist': grad_hist
}

iter =    1, stepsize = 5.663e-01, rdiff = 4.796e-01 , f = 0.0212893

Fista err = 9.137e-06; Subiter =  32; subproblem converged!

iter =    2, stepsize = 7.532e-01, rdiff = 2.064e-01 , f = -2.59625

Fista err = 9.988e-06; Subiter = 635; subproblem converged!

iter =    3, stepsize = 9.230e-01, rdiff = 5.318e-02 , f = -4.02128

Fista err = 9.725e-06; Subiter =   9; subproblem converged!

iter =    4, stepsize = 9.938e-01, rdiff = 4.001e-03 , f = -4.44041

Fista err = 9.632e-06; Subiter =   3; subproblem converged!

iter =    5, stepsize = 9.999e-01, rdiff = 7.149e-05 , f = -4.47488

Fista err = 7.394e-07; Subiter =   2; subproblem converged!

iter =    6, stepsize = 1.000e+00, rdiff = 1.770e-06 , f = -4.47509

Fista err = 2.271e-10; Subiter =   2; subproblem converged!

iter =    7, stepsize = 1.000e+00, rdiff = 1.061e-06 , f = -4.47509

Fista err = 2.271e-10; Subiter =   2; subproblem converged!

boo
Convergence achieved!
iter =    8, stepsize = 0.000e+00, rdiff = 0.000e+00,value=-4.4

In [10]:
with open(os.path.join(results_folder, 'portfolio', problem_name + '.pckl'), 'wb') as f:
    pickle.dump(results, f)